In [ ]:
import os
import math
import cv2
import numpy as np
import pandas as pd
import skimage.io
from PIL import Image
from tqdm.notebook import tqdm
import zipfile
import tifffile
import matplotlib.pyplot as plt

In [ ]:
# converting images to lists

Img = os.listdir('../input/hubmap-aaa6a05cc-256x256-tiles/images/Images_01')
# print(Img)

img = []

for filename in Img:
    if filename.endswith('.png'):
        img.append(filename)
    
img.sort()
img = img[:3000]

In [ ]:
# converting masks to lists

Msk = os.listdir('../input/hubmap-aaa6a05cc-256x256-tiles/masks/masks_01')

Mask = []

for filename in Msk:
    if filename.endswith('.png'):
        Mask.append(filename)
    
Mask.sort()
Mask = Mask[:3000]

In [ ]:
# converting list of Images and Mask into input 'shape'

y = np.zeros((3000, 256, 256 , 1), dtype=np.float32)
X = np.zeros((3000,256, 256, 1), dtype=np.float32)

n = 0
for file in tqdm(img , total = X.shape[0]):
    index = img.index(file)
    dir_img = os.path.join('../input/hubmap-aaa6a05cc-256x256-tiles/images/Images_01/', file)
    im = Image.open(dir_img)
    im = im.resize((256, 256))
    im = np.reshape(im.convert('L'), (256,256,1)) 
    X[n] = im
    
    mask = Mask[index]
    dir_mask = os.path.join('../input/hubmap-aaa6a05cc-256x256-tiles/masks/masks_01/', mask)
    mk = Image.open(dir_mask)
    mk = mk.resize((256, 256))
    mk = np.reshape(mk.convert('L'), (256,256,1)) 
    y[n] = mk
    n += 1

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
%env SM_FRAMEWORK=tf.keras

import keras
import tensorflow as tf
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from keras.models import model_from_json
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler

from keras.layers import Input, Conv2D, Reshape
from keras.models import Model
from keras import backend as K

In [ ]:
# Split train data into train and test sets

train_images, X_test, train_labels, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
#Preprocessing 'train_images' for segmentation model

BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)

train_images = preprocess_input(train_images)
train_labels = preprocess_input(train_labels)

In [ ]:
# Split train data into train and validation sets

X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels,
                                                  test_size=0.15, random_state=22)

#Preprocessing validation images for segmentation model
X_val = preprocess_input(X_val)
y_val = preprocess_input(y_val)

In [ ]:
X_train.shape , X_test.shape , X_val.shape

In [ ]:
# Model Building using U-net

from keras.layers import Reshape
N = X_train.shape[-1]

# create the base pre-trained model
base_model = Unet(backbone_name='resnet34', encoder_weights='imagenet')

# add a layer with defined input shape
input_base_model = Input(shape=(256, 256, N))

# add a convolution layer with input data
l1 = Conv2D(3, (1, 1))(input_base_model)

# defining output layer shape
out = base_model(l1)

# x1 = Conv2D(10, kernel_size =3,strides=2,padding = "same", activation="relu")(out)
# x1 =layers.BatchNormalization()(x1)

# x2= Conv2D(10, kernel_size=3,strides=2,padding = "same", activation="relu")(x1)
# x2 =layers.BatchNormalization()(x2)

# x3 = Conv2D(10, kernel_size=3,strides=2,padding = "same", activation="relu")(x2)
# x3 =layers.BatchNormalization()(x3)

# x4 = Conv2D(1, kernel_size=2,strides=2,padding = "same", activation="relu")(x3)

# x_out = Reshape((16,16))(x4)

model = Model(input_base_model, out, name=base_model.name)

In [ ]:
model.summary()

In [ ]:
# callbacks

lr_schedule = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 10))

initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)


early_stopping = keras.callbacks.EarlyStopping(patience=8 , verbose = 1)

model_checkpoint = keras.callbacks.ModelCheckpoint(
                   '/kaggle/working/best_cnn.h5', 
                   save_best_only=True)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule, amsgrad=False)

In [ ]:
# Defining metrics and loss

def dice_coefficient(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return numerator / (denominator + tf.keras.backend.epsilon())

def loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) - tf.math.log(dice_coefficient(y_true, y_pred) + tf.keras.backend.epsilon())

In [ ]:
# Model Compiling

model.compile(optimizer = optimizer, loss=binary_crossentropy, metrics=[dice_coefficient])

In [ ]:
# Data augmentation

datagen = ImageDataGenerator(rotation_range=8,
                             zoom_range=[0.95, 1.05],
                             height_shift_range=0.10,
                             shear_range=0.15)

In [ ]:
# Fit model in order to determine best learning rate 

history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                                  epochs=30, 
                                  validation_data=(X_val, y_val),
                                  callbacks=[early_stopping, model_checkpoint]
                                 )

In [ ]:
# history = model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test, y_test))

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.metrics_names

In [ ]:
dir(history)

In [ ]:
history.history

In [ ]:
dice_coefficient = history.history['dice_coefficient']
val_dice_coefficient = history.history['val_dice_coefficient']

train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:

plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.plot(dice_coefficient, label='Dice coefficient')
plt.plot(val_dice_coefficient, label='Validation Dice coefficient')
plt.legend()
plt.xlabel('Epochs')
plt.title('Epochs vs. Training and Validation Dice coefficient')
    
plt.subplot(1, 2, 2)
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend()
plt.xlabel('Epochs')
plt.title('Epochs vs. Training and Validation Loss')

plt.show()

In [ ]:
# Plot Learning Rate vs. Loss

plt.plot(history.epoch, history.history['loss'])
# plt.axis([1e-4, 1e-1, 0, 4])
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.show()